In [5]:
# Install necessary libraries
# !pip install googletrans==4.0.0-rc1 pandas scikit-learn tensorflow imbalanced-learn nlpaug

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, Bidirectional, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import re
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Load data from CSV file
df = pd.read_csv('171kReviewWithSentiment.csv')

# Keep only 'Summary' and 'Stance' columns
df = df[['Summary', 'Stance']]

# Drop rows where 'Summary' or 'Stance' is NaN
df = df.dropna(subset=['Summary', 'Stance'])

# Ensure all entries in 'Summary' are strings
df['Summary'] = df['Summary'].astype(str)

# For full review display without truncation
pd.set_option('display.max_colwidth', None)

# Print value counts of 'Stance' column
print(df['Stance'].value_counts())

# Text Preprocessing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure NLTK stopwords and WordNet lemmatizer are downloaded
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatize and remove stopwords
    return text

df['Summary'] = df['Summary'].apply(preprocess_text)

# Define back-translation function
def back_translate(text, src_language='en', mid_language='fr'):
    try:
        translated_text = translator.translate(text, src=src_language, dest=mid_language).text
        back_translated_text = translator.translate(translated_text, src=mid_language, dest=src_language).text
        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text

# Define function to augment minority class
def augment_minority_class(df, class_label, src_language='en', mid_language='fr'):
    minority_texts = df[df['Stance'] == class_label]['Summary'].tolist()
    augmented_texts = [back_translate(text, src_language, mid_language) for text in minority_texts]
    augmented_labels = [class_label] * len(augmented_texts)
    return pd.DataFrame({'Summary': augmented_texts, 'Stance': augmented_labels})

# Define threshold for minority class
threshold = 100  # Adjust based on your needs

# Identify minority classes
class_counts = df['Stance'].value_counts()
minority_classes = class_counts[class_counts < threshold].index.tolist()

# Apply back-translation for all minority classes
augmented_df_list = [augment_minority_class(df, cls) for cls in minority_classes]
df_augmented = pd.concat([df] + augmented_df_list, ignore_index=True)

# Text Vectorization for ML models
tfidf = TfidfVectorizer(max_df=0.7)
X = tfidf.fit_transform(df_augmented['Summary'])
y = df_augmented['Stance']

# Handle class imbalance using SMOTE for traditional ML models
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Split dataset for ML models
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Train and evaluate ML models
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train_ml, y_train_ml)
    y_pred = model.predict(X_test_ml)
    accuracy = accuracy_score(y_test_ml, y_pred)
    report = classification_report(y_test_ml, y_pred)
    print(f"{name} Results:")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)

# Tokenizer and padding for deep learning models
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_augmented['Summary'])
X_seq = tokenizer.texts_to_sequences(df_augmented['Summary'])
X_pad = pad_sequences(X_seq, maxlen=100)
y_encoded = pd.get_dummies(df_augmented['Stance']).values

# Split dataset for deep learning models
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# Define and train CNN model
cnn_model = Sequential([
    Embedding(5000, 128, input_length=100),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')
])
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

cnn_y_pred = cnn_model.predict(X_test_dl)
cnn_y_pred_labels = cnn_y_pred.argmax(axis=1)
y_test_dl_labels = y_test_dl.argmax(axis=1)
cnn_accuracy = accuracy_score(y_test_dl_labels, cnn_y_pred_labels)
cnn_report = classification_report(y_test_dl_labels, cnn_y_pred_labels)
print("CNN Results:")
print(f"Accuracy: {cnn_accuracy}")
print("Classification Report:")
print(cnn_report)

# Define and train LSTM model
lstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(y_encoded.shape[1], activation='softmax')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

lstm_y_pred = lstm_model.predict(X_test_dl)
lstm_y_pred_labels = lstm_y_pred.argmax(axis=1)
lstm_accuracy = accuracy_score(y_test_dl_labels, lstm_y_pred_labels)
lstm_report = classification_report(y_test_dl_labels, lstm_y_pred_labels)
print("LSTM Results:")
print(f"Accuracy: {lstm_accuracy}")
print("Classification Report:")
print(lstm_report)

# Define and train BiLSTM model
bilstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(y_encoded.shape[1], activation='softmax')
])
bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bilstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

bilstm_y_pred = bilstm_model.predict(X_test_dl)
bilstm_y_pred_labels = bilstm_y_pred.argmax(axis=1)
bilstm_accuracy = accuracy_score(y_test_dl_labels, bilstm_y_pred_labels)
bilstm_report = classification_report(y_test_dl_labels, bilstm_y_pred_labels)
print("BiLSTM Results:")
print(f"Accuracy: {bilstm_accuracy}")
print("Classification Report:")
print(bilstm_report)

# Compare all models
accuracy_scores_all = {
    'Naive Bayes': accuracy_score(y_test_ml, models['Naive Bayes'].predict(X_test_ml)),
    'SVM': accuracy_score(y_test_ml, models['SVM'].predict(X_test_ml)),
    'Random Forest': accuracy_score(y_test_ml, models['Random Forest'].predict(X_test_ml)),
    'CNN': cnn_accuracy,
    'LSTM': lstm_accuracy,
    'BiLSTM': bilstm_accuracy
}

best_model = max(accuracy_scores_all, key=accuracy_scores_all.get)
print(f"\nThe best model based on accuracy is: {best_model} with accuracy {accuracy_scores_all[best_model]}")


Stance
supportive    25913
neutral       25790
oppose        25001
Name: count, dtype: int64


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Moneykicks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Moneykicks\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Naive Bayes Results:
Accuracy: 0.792191921790584
Classification Report:
              precision    recall  f1-score   support

     neutral       0.81      0.67      0.73      5279
      oppose       0.75      0.82      0.78      5160
  supportive       0.82      0.90      0.86      5109

    accuracy                           0.79     15548
   macro avg       0.79      0.79      0.79     15548
weighted avg       0.79      0.79      0.79     15548

SVM Results:
Accuracy: 0.8040905582711603
Classification Report:
              precision    recall  f1-score   support

     neutral       0.77      0.75      0.76      5279
      oppose       0.78      0.78      0.78      5160
  supportive       0.86      0.88      0.87      5109

    accuracy                           0.80     15548
   macro avg       0.80      0.80      0.80     15548
weighted avg       0.80      0.80      0.80     15548

Random Forest Results:
Accuracy: 0.8175971186004631
Classification Report:
              precision   

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 25s - 26ms/step - accuracy: 0.7927 - loss: 0.5168 - val_accuracy: 0.8153 - val_loss: 0.4604
Epoch 2/5
959/959 - 23s - 24ms/step - accuracy: 0.8449 - loss: 0.4028 - val_accuracy: 0.8196 - val_loss: 0.4539
Epoch 3/5
959/959 - 24s - 25ms/step - accuracy: 0.8727 - loss: 0.3391 - val_accuracy: 0.8193 - val_loss: 0.4783
Epoch 4/5
959/959 - 23s - 24ms/step - accuracy: 0.8916 - loss: 0.2947 - val_accuracy: 0.8174 - val_loss: 0.5017
Epoch 5/5
959/959 - 24s - 25ms/step - accuracy: 0.9010 - loss: 0.2650 - val_accuracy: 0.8146 - val_loss: 0.5718
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
CNN Results:
Accuracy: 0.8146144319144776
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.74      0.77      5257
           1       0.80      0.80      0.80      4976
           2       0.84      0.91      0.87      5108

    accuracy                           0.81     15341
   macro avg       0.81      0.82      0.81     15341
weighted avg      

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 66s - 69ms/step - accuracy: 0.7718 - loss: 0.5633 - val_accuracy: 0.8048 - val_loss: 0.4829
Epoch 2/5
959/959 - 63s - 65ms/step - accuracy: 0.8215 - loss: 0.4551 - val_accuracy: 0.8091 - val_loss: 0.4732
Epoch 3/5
959/959 - 66s - 69ms/step - accuracy: 0.8338 - loss: 0.4257 - val_accuracy: 0.8123 - val_loss: 0.4686
Epoch 4/5
959/959 - 64s - 67ms/step - accuracy: 0.8434 - loss: 0.4041 - val_accuracy: 0.8128 - val_loss: 0.4745
Epoch 5/5
959/959 - 70s - 73ms/step - accuracy: 0.8489 - loss: 0.3873 - val_accuracy: 0.8172 - val_loss: 0.4723
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step
LSTM Results:
Accuracy: 0.8172218238706733
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.77      5257
           1       0.79      0.80      0.80      4976
           2       0.86      0.90      0.88      5108

    accuracy                           0.82     15341
   macro avg       0.82      0.82      0.82     15341
weighted avg    

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 108s - 113ms/step - accuracy: 0.7697 - loss: 0.5666 - val_accuracy: 0.8050 - val_loss: 0.4812
Epoch 2/5
959/959 - 101s - 105ms/step - accuracy: 0.8207 - loss: 0.4560 - val_accuracy: 0.8112 - val_loss: 0.4678
Epoch 3/5
959/959 - 100s - 104ms/step - accuracy: 0.8338 - loss: 0.4260 - val_accuracy: 0.8137 - val_loss: 0.4666
Epoch 4/5
959/959 - 100s - 104ms/step - accuracy: 0.8430 - loss: 0.4050 - val_accuracy: 0.8138 - val_loss: 0.4689
Epoch 5/5
959/959 - 100s - 104ms/step - accuracy: 0.8491 - loss: 0.3880 - val_accuracy: 0.8164 - val_loss: 0.4726
480/480 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step
BiLSTM Results:
Accuracy: 0.8164396062838146
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.77      5257
           1       0.79      0.80      0.80      4976
           2       0.86      0.90      0.88      5108

    accuracy                           0.82     15341
   macro avg       0.82      0.82      0.82     15341
weig

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Load NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load the original dataset
df_original = pd.read_csv('171kReviewWithSentiment.csv')

# Keep 'reviewerName' and 'reviewText' columns
df = df_original[['reviewerName', 'reviewText']]

# Drop rows where any of the required columns are NaN
df = df.dropna(subset=['reviewerName', 'reviewText'])

# Ensure all entries in 'reviewText' are strings
df['reviewText'] = df['reviewText'].astype(str)

# For full review display without truncation
pd.set_option('display.max_colwidth', None)

# Text Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatize and remove stopwords
    return text

df['reviewText'] = df['reviewText'].apply(preprocess_text)

# Function to get user reviews
def get_user_reviews(df, user_name):
    """Extract reviews for a specific user."""
    return df[df['reviewerName'] == user_name]['reviewText'].tolist()

# Function to compute similarity
def compute_similarity(user1_reviews, user2_reviews, vectorizer):
    """Compute cosine similarity between reviews of two users."""
    # Combine reviews from both users
    combined_reviews = user1_reviews + user2_reviews
    
    # Transform reviews to TF-IDF vectors
    vectors = vectorizer.transform(combined_reviews)
    
    # Compute cosine similarity between user 1 and user 2
    similarity_matrix = cosine_similarity(vectors[:len(user1_reviews)], vectors[len(user1_reviews):])
    
    # Return the average similarity score
    return similarity_matrix.mean()

# Define your users
user1 = '53rdcard'
user2 = 'Aaron'

# Get reviews for each user
user1_reviews = get_user_reviews(df, user1)
user2_reviews = get_user_reviews(df, user2)

# Vectorize the reviews using TF-IDF
tfidf = TfidfVectorizer(max_df=0.7)
tfidf.fit(df['reviewText'])  # Fit on all reviews in the dataset

# Compute similarity score
similarity_score = compute_similarity(user1_reviews, user2_reviews, tfidf)
print(f"Similarity Score between {user1} and {user2}: {similarity_score}")


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, Bidirectional, SpatialDropout1D
from imblearn.over_sampling import SMOTE
import re
from googletrans import Translator 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Load NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize Translator
translator = Translator()

# Load the original dataset
df_original = pd.read_csv('171kReviewWithSentiment.csv')

# Keep  'Summary', and 'Stance' columns
df = df_original[['Summary', 'Stance']]

# Drop rows where any of the required columns are NaN
df = df.dropna(subset=[ 'Summary', 'Stance'])

# Ensure all entries in 'Summary' are strings
df['Summary'] = df['Summary'].astype(str)

# For full review display without truncation
pd.set_option('display.max_colwidth', None)

# Print value counts of 'Stance' column
print(df['Stance'].value_counts())

# Text Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatize and remove stopwords
    return text

df['Summary'] = df['Summary'].apply(preprocess_text)

# Define back-translation function
def back_translate(text, src_language='en', mid_language='fr'):
    try:
        translated_text = translator.translate(text, src=src_language, dest=mid_language).text
        back_translated_text = translator.translate(translated_text, src=mid_language, dest=src_language).text
        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text

# Define function to augment minority class
def augment_minority_class(df, class_label, src_language='en', mid_language='fr'):
    minority_texts = df[df['Stance'] == class_label]['Summary'].tolist()
    augmented_texts = [back_translate(text, src_language, mid_language) for text in minority_texts]
    augmented_labels = [class_label] * len(augmented_texts)
    return pd.DataFrame({'Summary': augmented_texts, 'Stance': augmented_labels})

# Define threshold for minority class
threshold = 100  # Adjust based on your needs

# Identify minority classes
class_counts = df['Stance'].value_counts()
minority_classes = class_counts[class_counts < threshold].index.tolist()

# Apply back-translation for all minority classes
augmented_df_list = [augment_minority_class(df, cls) for cls in minority_classes]
df_augmented = pd.concat([df] + augmented_df_list, ignore_index=True)

# Text Vectorization for ML models
tfidf = TfidfVectorizer(max_df=0.7)
X = tfidf.fit_transform(df_augmented['Summary'])
y = df_augmented['Stance']

# Handle class imbalance using SMOTE for traditional ML models
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Split dataset for ML models
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Train and evaluate ML models
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear', probability=True),  # Set probability=True for SVM
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Confidence threshold
confidence_threshold = 0.8

def trust_based_accuracy(y_true, y_pred, confidence_scores, threshold):
    mask = confidence_scores >= threshold
    y_true_trust = y_true[mask]
    y_pred_trust = y_pred[mask]
    if len(y_true_trust) == 0:
        return None  # No predictions above the threshold
    return accuracy_score(y_true_trust, y_pred_trust)

for name, model in models.items():
    model.fit(X_train_ml, y_train_ml)
    y_pred = model.predict(X_test_ml)
    y_probs = model.predict_proba(X_test_ml)
    confidence_scores = y_probs.max(axis=1)
    
    # Calculate standard accuracy
    accuracy = accuracy_score(y_test_ml, y_pred)
    report = classification_report(y_test_ml, y_pred)
    print(f"{name} Results:")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)
    
    # Calculate trust-based accuracy
    trust_accuracy = trust_based_accuracy(y_test_ml, y_pred, confidence_scores, confidence_threshold)
    if trust_accuracy is not None:
        print(f"Trust-Based Accuracy for {name}: {trust_accuracy}")
    else:
        print(f"No predictions above the confidence threshold for {name}")

# Tokenizer and padding for deep learning models
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_augmented['Summary'])
X_seq = tokenizer.texts_to_sequences(df_augmented['Summary'])
X_pad = pad_sequences(X_seq, maxlen=100)
y_encoded = pd.get_dummies(df_augmented['Stance']).values

# Split dataset for deep learning models
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# Define and train CNN model
cnn_model = Sequential([
    Embedding(5000, 128, input_length=100),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')
])
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

cnn_y_pred = cnn_model.predict(X_test_dl)
cnn_y_pred_labels = cnn_y_pred.argmax(axis=1)
y_test_dl_labels = y_test_dl.argmax(axis=1)
cnn_accuracy = accuracy_score(y_test_dl_labels, cnn_y_pred_labels)
cnn_report = classification_report(y_test_dl_labels, cnn_y_pred_labels)
print("CNN Results:")
print(f"Accuracy: {cnn_accuracy}")
print("Classification Report:")
print(cnn_report)

# Calculate confidence scores for CNN
cnn_confidence_scores = cnn_y_pred.max(axis=1)

# Calculate trust-based accuracy for CNN
cnn_trust_accuracy = trust_based_accuracy(y_test_dl_labels, cnn_y_pred_labels, cnn_confidence_scores, confidence_threshold)
if cnn_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for CNN: {cnn_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for CNN")

# Define and train LSTM model
lstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(y_encoded.shape[1], activation='softmax')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

lstm_y_pred = lstm_model.predict(X_test_dl)
lstm_y_pred_labels = lstm_y_pred.argmax(axis=1)
lstm_accuracy = accuracy_score(y_test_dl_labels, lstm_y_pred_labels)
lstm_report = classification_report(y_test_dl_labels, lstm_y_pred_labels)
print("LSTM Results:")
print(f"Accuracy: {lstm_accuracy}")
print("Classification Report:")
print(lstm_report)

# Calculate confidence scores for LSTM
lstm_confidence_scores = lstm_y_pred.max(axis=1)

# Calculate trust-based accuracy for LSTM
lstm_trust_accuracy = trust_based_accuracy(y_test_dl_labels, lstm_y_pred_labels, lstm_confidence_scores, confidence_threshold)
if lstm_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for LSTM: {lstm_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for LSTM")

# Define and train BiLSTM model
bilstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(y_encoded.shape[1], activation='softmax')
])
bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bilstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

bilstm_y_pred = bilstm_model.predict(X_test_dl)
bilstm_y_pred_labels = bilstm_y_pred.argmax(axis=1)
bilstm_accuracy = accuracy_score(y_test_dl_labels, bilstm_y_pred_labels)
bilstm_report = classification_report(y_test_dl_labels, bilstm_y_pred_labels)
print("BiLSTM Results:")
print(f"Accuracy: {bilstm_accuracy}")
print("Classification Report:")
print(bilstm_report)

# Calculate confidence scores for BiLSTM
bilstm_confidence_scores = bilstm_y_pred.max(axis=1)

# Calculate trust-based accuracy for BiLSTM
bilstm_trust_accuracy = trust_based_accuracy(y_test_dl_labels, bilstm_y_pred_labels, bilstm_confidence_scores, confidence_threshold)
if bilstm_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for BiLSTM: {bilstm_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for BiLSTM")

# Compare all models
accuracy_scores_all = {
    'Naive Bayes': accuracy_score(y_test_ml, models['Naive Bayes'].predict(X_test_ml)),
    'SVM': accuracy_score(y_test_ml, models['SVM'].predict(X_test_ml)),
    'Random Forest': accuracy_score(y_test_ml, models['Random Forest'].predict(X_test_ml)),
    'CNN': cnn_accuracy,
    'LSTM': lstm_accuracy,
    'BiLSTM': bilstm_accuracy
}

best_model = max(accuracy_scores_all, key=accuracy_scores_all.get)
print(f"\nThe best model based on accuracy is: {best_model} with accuracy {accuracy_scores_all[best_model]}")

trust_accuracy_scores_all = {
    'Naive Bayes': trust_based_accuracy(y_test_ml, models['Naive Bayes'].predict(X_test_ml), models['Naive Bayes'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'SVM': trust_based_accuracy(y_test_ml, models['SVM'].predict(X_test_ml), models['SVM'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'Random Forest': trust_based_accuracy(y_test_ml, models['Random Forest'].predict(X_test_ml), models['Random Forest'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'CNN': cnn_trust_accuracy,
    'LSTM': lstm_trust_accuracy,
    'BiLSTM': bilstm_trust_accuracy
}

best_trust_model = max(trust_accuracy_scores_all, key=lambda k: trust_accuracy_scores_all[k] if trust_accuracy_scores_all[k] is not None else -1)
print(f"\nThe best model based on trust-based accuracy is: {best_trust_model} with trust-based accuracy {trust_accuracy_scores_all[best_trust_model]}")


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


Stance
supportive    25913
neutral       25790
oppose        25001
Name: count, dtype: int64
Naive Bayes Results:
Accuracy: 0.792191921790584
Classification Report:
              precision    recall  f1-score   support

     neutral       0.81      0.67      0.73      5279
      oppose       0.75      0.82      0.78      5160
  supportive       0.82      0.90      0.86      5109

    accuracy                           0.79     15548
   macro avg       0.79      0.79      0.79     15548
weighted avg       0.79      0.79      0.79     15548

Trust-Based Accuracy for Naive Bayes: 0.9522750598699966
SVM Results:
Accuracy: 0.8040905582711603
Classification Report:
              precision    recall  f1-score   support

     neutral       0.77      0.75      0.76      5279
      oppose       0.78      0.78      0.78      5160
  supportive       0.86      0.88      0.87      5109

    accuracy                           0.80     15548
   macro avg       0.80      0.80      0.80     15548
weight

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 23s - 25ms/step - accuracy: 0.7905 - loss: 0.5185 - val_accuracy: 0.8162 - val_loss: 0.4573
Epoch 2/5
959/959 - 24s - 25ms/step - accuracy: 0.8443 - loss: 0.4042 - val_accuracy: 0.8213 - val_loss: 0.4488
Epoch 3/5
959/959 - 22s - 23ms/step - accuracy: 0.8711 - loss: 0.3415 - val_accuracy: 0.8149 - val_loss: 0.4861
Epoch 4/5
959/959 - 22s - 23ms/step - accuracy: 0.8900 - loss: 0.2957 - val_accuracy: 0.8184 - val_loss: 0.5179
Epoch 5/5
959/959 - 23s - 24ms/step - accuracy: 0.9019 - loss: 0.2654 - val_accuracy: 0.8159 - val_loss: 0.5351
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
CNN Results:
Accuracy: 0.8158529430936705
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.77      0.78      5257
           1       0.82      0.77      0.79      4976
           2       0.84      0.91      0.87      5108

    accuracy                           0.82     15341
   macro avg       0.82      0.82      0.81     15341
weighted avg      

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 66s - 69ms/step - accuracy: 0.7689 - loss: 0.5684 - val_accuracy: 0.8066 - val_loss: 0.4805
Epoch 2/5
959/959 - 72s - 75ms/step - accuracy: 0.8205 - loss: 0.4584 - val_accuracy: 0.8116 - val_loss: 0.4694
Epoch 3/5
959/959 - 66s - 69ms/step - accuracy: 0.8335 - loss: 0.4283 - val_accuracy: 0.8138 - val_loss: 0.4661
Epoch 4/5
959/959 - 71s - 74ms/step - accuracy: 0.8409 - loss: 0.4073 - val_accuracy: 0.8151 - val_loss: 0.4669
Epoch 5/5
959/959 - 66s - 69ms/step - accuracy: 0.8473 - loss: 0.3904 - val_accuracy: 0.8179 - val_loss: 0.4689
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step
LSTM Results:
Accuracy: 0.8178736718597223
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.77      5257
           1       0.80      0.79      0.80      4976
           2       0.85      0.91      0.88      5108

    accuracy                           0.82     15341
   macro avg       0.82      0.82      0.82     15341
weighted avg    

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


959/959 - 113s - 118ms/step - accuracy: 0.7664 - loss: 0.5708 - val_accuracy: 0.8039 - val_loss: 0.4871
Epoch 2/5
959/959 - 108s - 112ms/step - accuracy: 0.8210 - loss: 0.4589 - val_accuracy: 0.8129 - val_loss: 0.4715
Epoch 3/5
959/959 - 103s - 108ms/step - accuracy: 0.8334 - loss: 0.4282 - val_accuracy: 0.8142 - val_loss: 0.4669
Epoch 4/5
959/959 - 105s - 109ms/step - accuracy: 0.8407 - loss: 0.4082 - val_accuracy: 0.8117 - val_loss: 0.4719
Epoch 5/5
959/959 - 106s - 111ms/step - accuracy: 0.8478 - loss: 0.3890 - val_accuracy: 0.8138 - val_loss: 0.4682
480/480 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step
BiLSTM Results:
Accuracy: 0.8137670295287139
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.75      0.77      5257
           1       0.80      0.79      0.79      4976
           2       0.84      0.91      0.88      5108

    accuracy                           0.81     15341
   macro avg       0.81      0.81      0.81     15341
wei